### Post order

In [26]:
import asyncio
import xrpl
import os
from xrpl.wallet import Wallet
from xrpl.models.transactions import OfferCreate
from xrpl.transaction import submit_and_wait
from xrpl.ledger import get_latest_validated_ledger_sequence
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.account import get_next_valid_seq_number



async def place_order(seed, taker_gets, taker_pays):
    client = AsyncJsonRpcClient("https://xrplcluster.com")  # Mainnet
    client = AsyncJsonRpcClient("https://s.altnet.rippletest.net:51234")  # Testnet

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    print(wallet.classic_address)
    # Get the latest ledger index
    latest_ledger = await asyncio.to_thread(get_latest_validated_ledger_sequence, client)
    sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCreate transaction
    offer_tx = OfferCreate(
        account=wallet.classic_address,
        taker_gets=taker_gets,  # What you are selling
        taker_pays=taker_pays,  # What you want in return
        sequence=sequence,  # Ensure sequence is set
        last_ledger_sequence=latest_ledger + 5  # Valid for the next 5 ledgers
    )

    # Submit the transaction and wait for a result
    response = await asyncio.to_thread(submit_and_wait, offer_tx, client, wallet)

    # Display transaction response
    print("\nTransaction Response:")
    print(response)

     
# 🔥 Example: Selling 10 XRP for 10 USD issued by an issuer
seed = os.getenv("XRPL_ACCOUNT_SECRET")  # Replace with your real wallet seed
taker_gets = "10000000"  # Amount in drops (10 XRP = 10,000,000 drops)
taker_pays = {
    "currency": "USD",
    "issuer": "rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B",
    "value": "27.12345"  # Receiving 10 USD
}

# Run the async function
await place_order(seed, taker_gets, taker_pays)

rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY

Transaction Response:
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'close_time_iso': '2025-02-10T19:12:20Z', 'ctid': 'C59B344600320000', 'hash': '24F3F3D24274D5FDCE0EF360F58AA194199C97DA1A73B558907EBA5B719C8297', 'ledger_hash': 'DF0DC27A750D524C4FA756CB505A9A4248A43C0A6CD0DC583858B30397179838', 'ledger_index': 94057542, 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'Balance': '500153210', 'Flags': 0, 'OwnerCount': 12, 'Sequence': 366}, 'LedgerEntryType': 'AccountRoot', 'LedgerIndex': '621598FB064E9B32493510B18D16957679C5BCC6D47554A500480972D8D9F151', 'PreviousFields': {'Balance': '500153220', 'OwnerCount': 11, 'Sequence': 365}, 'PreviousTxnID': 'B7F716AC10947AAFFDF802E922CF0BFF6EF4DEC96F7336B94059C4D03904DACC', 'PreviousTxnLgrSeq': 94056239}}, {'CreatedNode': {'LedgerEntryType': 'Offer', 'LedgerIndex': '6E4C28F6008E4D580B79CC5FD9D2873F855064BD7D7484CB6D738526A1869A50', 'N

### Delete order

In [27]:
import asyncio
import xrpl
from xrpl.wallet import Wallet
from xrpl.models.transactions import OfferCancel
from xrpl.transaction import submit_and_wait
from xrpl.account import get_next_valid_seq_number

async def cancel_order(wallet_seed, offer_sequence):
    client = xrpl.clients.JsonRpcClient("https://xrplcluster.com")  # Mainnet
    client = xrpl.clients.JsonRpcClient("https://s.altnet.rippletest.net:51234")  # Testnet

    # Load the wallet
    wallet = Wallet.from_seed(wallet_seed, algorithm="secp256k1")  # Ensure correct key type

    # Fetch the correct sequence number
    account_sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCancel transaction
    cancel_tx = OfferCancel(
        account=wallet.classic_address,
        offer_sequence=offer_sequence,
        sequence=account_sequence
    )

    # Submit the transaction and wait for confirmation
    response = await asyncio.to_thread(submit_and_wait, cancel_tx, client, wallet)

    # Display transaction response
    print("\nTransaction Response:")
    print(response)

# 🔥 Example: Cancel an order with sequence number 123456
wallet_seed = os.getenv("XRPL_ACCOUNT_SECRET")  # Replace with your real wallet seed
offer_sequence = 365  # Replace with the sequence number of the order to cancel


await cancel_order(wallet_seed, offer_sequence)


Transaction Response:
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'close_time_iso': '2025-02-10T19:13:11Z', 'ctid': 'C59B345300000000', 'hash': '8F8FC2001C1B3152023621585B5B261F0CF78EFF29D7121304DCBD881C531591', 'ledger_hash': '9620BF52B8A426F6972186761344A9208E9FD37B0EC995F72B4B49576498BCCE', 'ledger_index': 94057555, 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'Balance': '500153200', 'Flags': 0, 'OwnerCount': 11, 'Sequence': 367}, 'LedgerEntryType': 'AccountRoot', 'LedgerIndex': '621598FB064E9B32493510B18D16957679C5BCC6D47554A500480972D8D9F151', 'PreviousFields': {'Balance': '500153210', 'OwnerCount': 12, 'Sequence': 366}, 'PreviousTxnID': '24F3F3D24274D5FDCE0EF360F58AA194199C97DA1A73B558907EBA5B719C8297', 'PreviousTxnLgrSeq': 94057542}}, {'DeletedNode': {'FinalFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'BookDirectory': 'DFA3B6DDAB58C7E8E5D944E736DA4B7046C30E4F460FD9DE4F09A2DD083BBA00'